In [36]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [37]:
user_content = pd.read_csv("data/azure/VAS_user_content.csv")

In [38]:
vas_scored_df = pd.read_csv("data/azure/scored/vas_scored.csv")

### Applying Business rules

In [39]:
free_packages = [ 'V_Call Forwarding Offline','V_SLT Voice Service',
       'V_Anonymous call barring', 'V_Call back on busy',
       'V_CLI presentation in call waiting', 'V_Incoming Call Memory',
       'V_DQ Contact Details', 
       'V_Call Forwarding by time', 
       'V_Outgoing Call Memory', 
       'V_Incoming Call Transfer', 
       'V_Call holding', 'E_PeoTV Initiation',
       'V_Call Transfer Three way',
       'V_Call Forwarding-Immediate', 'V_Call Forwarding-On Busy',
       'V_Call Waiting', 'V_Call Forwarding-No Answer',
       'V_Call Forwarding', 'V_Absentee service',
       'V_Call Forwarding - On Busy', 'V_Call Forwarding - No Answer',
       'V_Abbreviated Dialing', 'P_Incoming Call Memory', 'P_Call Forwarding by time',
       'P_CLI presentation in call waiting', 
       'P_Anonymous call barring', 'P_Call back on busy',
       'P_Outgoing Call Memory', 'P_Call Forwarding Offline',
       'P_Call holding', 'P_Call Transfer Three way',
       'P_Incoming Call Transfer', 
       'DQ_Contact Details', 'P_SLT CLI',
       'V_Call Barring - Outgoing']

In [40]:
Peo_TV_Packages = ['PEO_SILVER', 'PEO_SILVER_PLUS', 'PEO_RELIGIOUS_PACKAGE',
       'NEW_SLT_STAFF_PACKAGE', 'PEO_UTHAYAM', 'PEO_TITANIUM',
       'PEO_UNNATHAM', 'PRANAMA', 'LOTUS', 'PEO_ENTERTAINMENT',
       'PEO_SILVER_PLUS_FTTH', 'SLTStaff_PEO_SILVER', 'PEO_PLATINUM',
       'DOUBLEPROMO', 'NEW_YEAR_TREAT', 'PEO_FAMILY', 'PEO_SILVER_FTTH',
       'BOC', 'PEO_GOLD', 'SLT_STAFF_PEO_UNNATHAM_FTTH',
       'SLT_STAFF_TRIPLE_PLAY']

In [41]:
Voice_VAS_Packages = ['V-Plus4_TeleLife', 'V-Plus1_TeleLife', 'V-Basic1_TeleLife',
       'V_Telehealth', 'V_Sisu Connect', 'V-Basic4_TeleLife',
       'V-Basic2_TeleLife', 'V_E-channeling Registration',
       'V_E-channeling Subscription', 'V_Double VAS Bundle', 'V_SLT CLI',
       'V_IDD', 'V_Single VAS Bundle', 'V_Cordeless Phone - Installment',
       'V_Hotline Service', 'V_Hotline-Immediate',
       'V_Short Message Service', 'V_Caller Line Identification',
       'V_Hotline-Time Delayed', 'V_Detailed Bill',
       'V_Railway Registration', 'V_Call Forwarding - Immediate',
       'V_One-Time Detailed Bill', 'V_Basic Phone Installment',
       'V_Installment', 'V_Referral offer Discount',
       'V_Call forwarding selective',
       'V_Caller name identificationPresentation',
       'V_Do not disturb service', 'V_Selective call waiting',
       'V_Selective call acceptance', 'V_Secretary service',
       'V_Password call barring (secret code)',
       'V_Selective call rejection', 'V_Caller name sending',
       'V_Complete call on no response', 'V_Triple VAS Bundle',
       'V_Basic Phone', 'V_Miscellaneous Service', 'V_Voice Startup Fee']

In [42]:
Peo_VAS_Packages = ['BB_PeoTVGO', 'E_SLT PeoTV Service', 'OTT_SLT PeoTV Go',
       'E_Video on Demand', 'PEO TV GO_OTT_A-la carte channels',
       'BB_PeoTVGO Revenue code', 'PeoTVGo_Channel Package']

In [43]:
BB_VAS_Packages = ['BB_SLT BroadBand Service', 'BB_Personal Storage',
       'BB_Detailed Reports ', 'BB_ Entertainment Unlimited',
       'BB_SLT Film Hall Service', 'BB_Report Subscription Charge',
       'BB Personal Storage', 'BB_YouTube Bundle', 'BB_Messenger Bundle']

In [44]:
vas_df = pd.merge(user_content[["event_source.hash","Sub_Type"]], vas_scored_df, left_on='event_source.hash', right_on="User", how='left', indicator='Exist')

In [45]:
vas_df = vas_df[(vas_df["Exist"]=="both")]

In [46]:
vas_df.reset_index(inplace=True)

In [47]:
vas_df.drop(["index","Predicted Rating 1","Predicted Rating 2","Predicted Rating 3","Predicted Rating 4","Predicted Rating 5"],axis=1, inplace=True)

In [49]:
# Applying Rules
def filter_vas(sub_type,vas):
    if not isinstance(sub_type, float):
        for fv in free_packages:
            if vas in fv:
                return
        if sub_type == "TRIPLE_PLAY":
            return vas
        for v in Voice_VAS_Packages:
            if vas in v:
                return vas
        if sub_type == "DOUBLE_PLAY_BV":
            for bv in BB_VAS_Packages:
                if vas in bv:
                    return vas
        if sub_type == "DOUBLE_PLAY_PV":
            for pv in Peo_VAS_Packages:
                if vas in pv:
                    return vas

In [50]:
recommender_cols = ["Recommended Item 1","Recommended Item 2","Recommended Item 3","Recommended Item 4","Recommended Item 5"]

In [52]:
for col in recommender_cols:
    vas_df[col] = vas_df.apply(lambda x: filter_vas(x["Sub_Type"],x[col]), axis=1)

In [53]:
vas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16760 entries, 0 to 16759
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   event_source.hash   16760 non-null  object  
 1   Sub_Type            16760 non-null  object  
 2   User                16760 non-null  object  
 3   Recommended Item 1  11137 non-null  object  
 4   Recommended Item 2  9768 non-null   object  
 5   Recommended Item 3  5387 non-null   object  
 6   Recommended Item 4  4042 non-null   object  
 7   Recommended Item 5  3168 non-null   object  
 8   Exist               16760 non-null  category
dtypes: category(1), object(8)
memory usage: 1.0+ MB


In [54]:
vas_df.drop(["Exist"],axis=1,inplace=True)

In [55]:
vas_df.to_csv("data/azure/scored/vas_scored_filtered.csv")